In [1]:
import os
import json
import cv2
import numpy as np
from pathlib import Path
from typing import List, Dict, Tuple
from PIL import Image
import shutil

In [3]:
class YOLOtoCOCOConverter:
    """Convert YOLO format dataset to COCO format"""
    
    def __init__(self, 
                 yolo_dataset_path: str,
                 output_path: str,
                 class_names: List[str],
                 thai_class_names: List[str] = None):
        """
        Args:
            yolo_dataset_path: Path to YOLO dataset (contains images/ and labels/)
            output_path: Output path for COCO format dataset
            class_names: List of class names in English
            thai_class_names: List of class names in Thai (optional)
        """
        self.yolo_path = Path(yolo_dataset_path)
        self.output_path = Path(output_path)
        self.class_names = class_names
        self.thai_class_names = thai_class_names or class_names
        
        # Create output directories
        self.output_path.mkdir(parents=True, exist_ok=True)
        (self.output_path / "images").mkdir(exist_ok=True)
        (self.output_path / "annotations").mkdir(exist_ok=True)
        
        print(f"🔄 Converting YOLO dataset from {yolo_dataset_path}")
        print(f"📁 Output will be saved to {output_path}")
        print(f"🏷️ Classes: {self.class_names}")
    
    def convert_bbox_yolo_to_coco(self, 
                                  yolo_bbox: List[float], 
                                  img_width: int, 
                                  img_height: int) -> List[float]:
        """
        Convert YOLO bbox format to COCO format
        
        YOLO: [x_center, y_center, width, height] (normalized 0-1)
        COCO: [x_min, y_min, width, height] (absolute pixels)
        """
        x_center, y_center, width, height = yolo_bbox
        
        # Convert to absolute coordinates
        x_center_abs = x_center * img_width
        y_center_abs = y_center * img_height
        width_abs = width * img_width
        height_abs = height * img_height
        
        # Convert to top-left corner format
        x_min = x_center_abs - (width_abs / 2)
        y_min = y_center_abs - (height_abs / 2)
        
        # Ensure coordinates are within image bounds
        x_min = max(0, min(x_min, img_width - 1))
        y_min = max(0, min(y_min, img_height - 1))
        width_abs = min(width_abs, img_width - x_min)
        height_abs = min(height_abs, img_height - y_min)
        
        return [x_min, y_min, width_abs, height_abs]
    
    def process_split(self, split_name: str) -> Dict:
        """Process train/val/test split"""
        
        print(f"\n📊 Processing {split_name} split...")
        
        # Paths
        images_dir = self.yolo_path / split_name / "images"
        labels_dir = self.yolo_path / split_name / "labels"
        
        if not images_dir.exists():
            print(f"⚠️ Images directory not found: {images_dir}")
            return {}
        
        if not labels_dir.exists():
            print(f"⚠️ Labels directory not found: {labels_dir}")
            return {}
        
        # COCO format structure
        coco_data = {
            "info": {
                "description": f"Indoor Object Detection Dataset - {split_name}",
                "version": "1.0",
                "year": 2024,
                "contributor": "Converted from YOLO format",
                "date_created": "2024-01-01"
            },
            "licenses": [
                {
                    "id": 1,
                    "name": "Unknown",
                    "url": ""
                }
            ],
            "images": [],
            "annotations": [],
            "categories": []
        }
        
        # Add categories
        for idx, (class_name, thai_name) in enumerate(zip(self.class_names, self.thai_class_names)):
            coco_data["categories"].append({
                "id": idx,
                "name": class_name,
                "thai_name": thai_name,
                "supercategory": "indoor_object"
            })
        
        # Copy images and process annotations
        output_images_dir = self.output_path / "images" / split_name
        output_images_dir.mkdir(parents=True, exist_ok=True)
        
        image_id = 1
        annotation_id = 1
        processed_count = 0
        error_count = 0
        
        # Get all image files
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        image_files = []
        for ext in image_extensions:
            image_files.extend(list(images_dir.glob(f"*{ext}")))
            image_files.extend(list(images_dir.glob(f"*{ext.upper()}")))
        
        print(f"📷 Found {len(image_files)} images")
        
        for image_path in image_files:
            try:
                # Load image to get dimensions
                image = Image.open(image_path)
                img_width, img_height = image.size
                
                # Copy image to output directory
                output_image_path = output_images_dir / image_path.name
                shutil.copy2(image_path, output_image_path)
                
                # Add image info
                coco_data["images"].append({
                    "id": image_id,
                    "file_name": f"{split_name}/{image_path.name}",
                    "width": img_width,
                    "height": img_height,
                    "license": 1
                })
                
                # Process corresponding label file
                label_path = labels_dir / f"{image_path.stem}.txt"
                
                if label_path.exists():
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        
                        try:
                            parts = line.split()
                            if len(parts) != 5:
                                print(f"⚠️ Invalid annotation format in {label_path}: {line}")
                                continue
                            
                            class_id = int(parts[0])
                            x_center = float(parts[1])
                            y_center = float(parts[2])
                            width = float(parts[3])
                            height = float(parts[4])
                            
                            # Validate class_id
                            if class_id >= len(self.class_names):
                                print(f"⚠️ Invalid class_id {class_id} in {label_path}")
                                continue
                            
                            # Convert bbox
                            yolo_bbox = [x_center, y_center, width, height]
                            coco_bbox = self.convert_bbox_yolo_to_coco(yolo_bbox, img_width, img_height)
                            
                            # Calculate area
                            area = coco_bbox[2] * coco_bbox[3]
                            
                            # Add annotation
                            coco_data["annotations"].append({
                                "id": annotation_id,
                                "image_id": image_id,
                                "category_id": class_id,
                                "bbox": coco_bbox,
                                "area": area,
                                "iscrowd": 0,
                                "segmentation": []
                            })
                            
                            annotation_id += 1
                            
                        except Exception as e:
                            print(f"⚠️ Error processing annotation {line} in {label_path}: {e}")
                            continue
                
                image_id += 1
                processed_count += 1
                
                if processed_count % 100 == 0:
                    print(f"  📊 Processed {processed_count} images...")
                
            except Exception as e:
                print(f"❌ Error processing {image_path}: {e}")
                error_count += 1
                continue
        
        # Save COCO annotations
        output_annotation_path = self.output_path / "annotations" / f"{split_name}.json"
        with open(output_annotation_path, 'w', encoding='utf-8') as f:
            json.dump(coco_data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ {split_name} conversion completed:")
        print(f"  📷 Images: {len(coco_data['images'])}")
        print(f"  🏷️ Annotations: {len(coco_data['annotations'])}")
        print(f"  ❌ Errors: {error_count}")
        print(f"  💾 Saved to: {output_annotation_path}")
        
        return coco_data
    
    def convert_dataset(self):
        """Convert entire dataset"""
        print("🚀 Starting YOLO to COCO conversion...")
        
        results = {}
        
        # Process each split
        for split in ['train', 'valid', 'test']:
            split_path = self.yolo_path / split
            if split_path.exists():
                results[split] = self.process_split(split)
            else:
                print(f"⚠️ Split {split} not found, skipping...")
        
        # Create summary
        self.create_summary(results)
        
        print("\n🎉 Conversion completed!")
        return results
    
    def create_summary(self, results: Dict):
        """Create conversion summary"""
        summary = {
            "dataset_info": {
                "name": "Indoor Object Detection Dataset",
                "format": "COCO",
                "converted_from": "YOLO",
                "classes": {
                    "count": len(self.class_names),
                    "names": self.class_names,
                    "thai_names": self.thai_class_names
                }
            },
            "splits": {}
        }
        
        total_images = 0
        total_annotations = 0
        
        for split_name, split_data in results.items():
            if split_data:
                split_summary = {
                    "images": len(split_data["images"]),
                    "annotations": len(split_data["annotations"]),
                    "categories": len(split_data["categories"])
                }
                summary["splits"][split_name] = split_summary
                total_images += split_summary["images"]
                total_annotations += split_summary["annotations"]
        
        summary["totals"] = {
            "images": total_images,
            "annotations": total_annotations
        }
        
        # Save summary
        summary_path = self.output_path / "dataset_summary.json"
        with open(summary_path, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        
        print(f"\n📋 Dataset Summary:")
        print(f"  📁 Total Images: {total_images}")
        print(f"  🏷️ Total Annotations: {total_annotations}")
        print(f"  📊 Splits: {list(results.keys())}")
        print(f"  💾 Summary saved to: {summary_path}")

def convert_indoor_dataset():
    """Convert indoor object detection dataset"""
    
    # Define class names
    class_names = [
        "door", "cabinetDoor", "refrigeratorDoor", "window", "chair",
        "table", "cabinet", "couch", "openedDoor", "pole"
    ]
    
    thai_class_names = [
        "ประตู", "ประตูตู้", "ประตูตู้เย็น", "หน้าต่าง", "เก้าอี้",
        "โต๊ะ", "ตู้", "โซฟา", "ประตูเปิด", "เสา"
    ]
    
    # Initialize converter
    converter = YOLOtoCOCOConverter(
        yolo_dataset_path="d:/4-1d/project/data/indoor-dataset",
        output_path="d:/4-1d/project/data/indoor-coco",
        class_names=class_names,
        thai_class_names=thai_class_names
    )
    
    # Convert dataset
    results = converter.convert_dataset()
    
    return results

if __name__ == "__main__":
    convert_indoor_dataset()

🔄 Converting YOLO dataset from d:/4-1d/project/data/indoor-dataset
📁 Output will be saved to d:/4-1d/project/data/indoor-coco
🏷️ Classes: ['door', 'cabinetDoor', 'refrigeratorDoor', 'window', 'chair', 'table', 'cabinet', 'couch', 'openedDoor', 'pole']
🚀 Starting YOLO to COCO conversion...

📊 Processing train split...
📷 Found 2024 images
  📊 Processed 100 images...
  📊 Processed 200 images...
  📊 Processed 300 images...
  📊 Processed 400 images...
  📊 Processed 500 images...
  📊 Processed 600 images...
  📊 Processed 700 images...
  📊 Processed 800 images...
  📊 Processed 900 images...
  📊 Processed 1000 images...
  📊 Processed 1100 images...
  📊 Processed 1200 images...
  📊 Processed 1300 images...
  📊 Processed 1400 images...
  📊 Processed 1500 images...
  📊 Processed 1600 images...
  📊 Processed 1700 images...
  📊 Processed 1800 images...
  📊 Processed 1900 images...
  📊 Processed 2000 images...
✅ train conversion completed:
  📷 Images: 2024
  🏷️ Annotations: 10984
  ❌ Errors: 0
  💾 S